In [1]:
import xml.etree.ElementTree as et
import numpy as np
import tensorflow as tf

In [2]:
labels = []
with open("data/reuters/labels.txt", "r") as ins:
    for line in ins:
        labels.append(line.strip("\n"))
# print(labels)
print("size labeles: ", len(labels))
label_size = len(labels)

data_lenght = 924
outputSize = 256

# Print out each one-hot encoded string for 3 species.
vocabulario_alpha = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's',
     't', 'u', 'v', 'w', 'x', 'y', 'z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ',', '.',
     ';', ':', '!', '?', '\'', '"', '|', '/', '_', '@', '#', '$', '%', '&', '^', '~', '+', '-', '*',
     '=', '<', '>', '(', ')', '[', ']', '{', '}']

voc_size = len(vocabulario_alpha)

def one_hot (text):
	matrix = np.array([np.zeros(voc_size, dtype=np.float64)])
	#print("text size: ", len(text))
	if len(text) > data_lenght:
		for i, val in enumerate(text):
			if i == data_lenght:
				break
			try:
				indice = vocabulario_alpha.index(val)
				array = np.zeros(voc_size)
				array[indice] = 1.0
				matrix = np.append(matrix, [array], axis = 0)
			except ValueError:
			    matrix = np.append(matrix, [np.zeros(voc_size)], axis = 0)
	else:
		j = 0
		for i, val in enumerate(text):
			j = i
			try:
				indice = vocabulario_alpha.index(val)
				array = np.zeros(voc_size)
				array[indice] = 1.0
				matrix = np.append(matrix, [array], axis = 0)
			except ValueError:
			    matrix = np.append(matrix, [np.zeros(voc_size)], axis = 0)
		#print("completar zeros: ", j)
		for i in range(j + 1, data_lenght):
			matrix = np.append(matrix, [np.zeros(voc_size)], axis = 0)
	matrix = np.delete(matrix, 0, 0)
	return matrix

def extractBody( text ):
   	for a_body in text.findall("BODY"):
   		if text.get("TYPE") is "BRIEF":
   			return None
   		return a_body
   	return ""
i = 0
train_data = []
train_data_labels = []

test_data = []
test_data_labels = []

average = 0

def load_text(i):
    extractlabels = False
    print "Extrayendo textos archivo " + str(i)
    e = ""
    if i < 10:
        e = et.parse('data/reuters/reut2-00' + str(i) + '.xml', et.XMLParser(encoding='ISO-8859-1')).getroot()
    else:
        e = et.parse('data/reuters/reut2-0' + str(i) + '.xml', et.XMLParser(encoding='ISO-8859-1')).getroot()
    for atype in e.findall('REUTERS'):
        # get body text
        #print("texto", i)
        matrix = []
        #if i == 2:
        #	break
        for a_text in atype.findall("TEXT"):
            body = extractBody(a_text)
            #if body != "":
            #    average = average + len(body.text)
            if body != "" and body != None:
                extractlabels = True
                #print(len(body.text))
                body = list(body.text)
                #print("size body: ", len(body))
                #print("size matrix: ", len(matrix))
                #print("size first element: ", len(matrix[0]))
                #print(matrix)
                #data.append(matrix)
                i = i + 1
            if extractlabels == True:
                #print("buscando etiquetas")
                label = np.zeros(label_size)
                total_etiquetas = 0
                for a_topic in atype.findall("TOPICS"):
                    for a_d in a_topic.findall("D"):
                        #print(a_d.text)
                        try:
                            indice = labels.index(a_d.text)
                            #print("indice: ", indice)
                            total_etiquetas = total_etiquetas + 1
                            label[indice] = 1.0
                        except ValueError:
                            l = 0
                for a_topic in atype.findall("PLACES"):
                    for a_d in a_topic.findall("D"):
                        #print(a_d.text)
                        try:
                            indice = labels.index(a_d.text)
                            #print("indice: ", indice)
                            total_etiquetas = total_etiquetas + 1
                            label[indice] = 1.0
                        except ValueError:
                            l = 0
                for a_topic in atype.findall("PEOPLE"):
                    for a_d in a_topic.findall("D"):
                        #print(a_d.text)
                        try:
                            indice = labels.index(a_d.text)
                            total_etiquetas = total_etiquetas + 1
                            label[indice] = 1.0
                        except ValueError:
                            l = 0
                for a_topic in atype.findall("ORGS"):
                    for a_d in a_topic.findall("D"):
                        #print(a_d.text)
                        try:
                            indice = labels.index(a_d.text)
                            total_etiquetas = total_etiquetas + 1
                            label[indice] = 1.0
                        except ValueError:
                            l = 0
                for a_topic in atype.findall("EXCHANGES"):
                    for a_d in a_topic.findall("D"):
                        #print(a_d.text)
                        try:
                            indice = labels.index(a_d.text)
                            total_etiquetas = total_etiquetas + 1
                            label[indice] = 1.0
                        except ValueError:
                            l = 0
                
                if total_etiquetas != 0:
                    
                    if atype.get("LEWISSPLIT") == "TRAIN":
                        train_data_labels.append(label)
                        matrix = one_hot(body)
                        train_data.append(matrix)
                    if atype.get("LEWISSPLIT") == "TEST":
                        test_data_labels.append(label)
                        matrix = one_hot(body)
                        test_data.append(matrix)
                    extractlabels = False
                else:
                    extractlabels = False
                    #data = np.delete(data, len(data) - 1, 0)

print("Textos Train: ", len(train_data))
print("Etiquetas Train: ", len(train_data_labels))

print("Textos Test: ", len(test_data))
print("Etiquetas Test: ", len(test_data_labels))
#print(data_labels[0])
#for i in range(0,len(data)):
	#print("x: ", len(data[i]))
	#print("y: ", len(data[i][0]))
	#print(data[i])
#print(data[0])

# print("end")


('size labeles: ', 672)
('Textos Train: ', 0)
('Etiquetas Train: ', 0)
('Textos Test: ', 0)
('Etiquetas Test: ', 0)


In [3]:
# Parameters
learning_rate = 0.001
training_iters = 100
batch_size = 128
display_step = 10

# Network Parameters
n_input = voc_size * data_lenght # vocabulary * text character size (img shape: l * 924)
n_classes = label_size # reuters total classes
dropout = 0.5 # Dropout, probability to keep units

# tf Graph input
x = tf.placeholder(tf.float32, [1, n_input])
y = tf.placeholder(tf.float32, [1, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [4]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([7, voc_size, 256])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([7, 256, 256])),
    'wc3': tf.Variable(tf.random_normal([3, 256, 256])),
    'wc4': tf.Variable(tf.random_normal([3, 256, 256])),
    'wc5': tf.Variable(tf.random_normal([5, 256, 256])),
    'wc6': tf.Variable(tf.random_normal([3, 256, 256])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([30*256, 1024])),
    'wd2': tf.Variable(tf.random_normal([1024, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, label_size]))
}
biases = {
    'bc1': tf.Variable(tf.random_normal([256])),
    'bc2': tf.Variable(tf.random_normal([256])),
    'bc3': tf.Variable(tf.random_normal([256])),
    'bc4': tf.Variable(tf.random_normal([256])),
    'bc5': tf.Variable(tf.random_normal([256])),
    'bc6': tf.Variable(tf.random_normal([256])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'bd2': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([label_size]))
}

In [4]:
print(data[0])
da = np.asarray(data[0]).reshape(-1)
print(da)
da = np.asarray(da).reshape(1, data_lenght, voc_size)
print(da)
input_data = tf.Variable(da.astype(np.float32))
print(input_data)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
[ 0.  0.  0. ...,  0.  0.  0.]
[[[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]]
Tensor("Variable_18/read:0", shape=(1, 924, 66), dtype=float32)


In [5]:
# Create some wrappers for simplicity
def conv1d(x, filters, bias, strides=1):
    # Conv1D wrapper, with bias and relu activation
    # x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.conv1d(x, filters, strides, padding='VALID')
    x = tf.nn.bias_add(x, bias)
    return tf.nn.relu(x)

def maxpool1d(x, lenght, output, k=2):
    # MaxPool2D wrapper
    x = tf.reshape(x, shape=[-1, 1, lenght, output])
    
    x = tf.nn.max_pool(x, ksize=[1, 1, k, 1], strides=[1, 1, k, 1],
                          padding='VALID')
    x = tf.reshape(x, shape=[-1, lenght / k, output])
    return x

def conv_net(x, weights, biases, dropout):
    #print(x)
    da = tf.reshape(x, [-1])
    #print(da)
    input_data = tf.reshape(x, [1, data_lenght, voc_size])
    #print(input_data)
    #input_data = tf.Variable(da.astype(np.float32))
    #print(input_data)
    conv1 = conv1d(input_data, weights['wc1'], biases['bc1'], strides=1)
    #print(conv1)
    conv1 = maxpool1d(conv1, 918, 256, 3)
    #print(conv1)

    conv2 = conv1d(conv1, weights['wc2'], biases['bc2'], strides=1)
    #print(conv2)
    conv2 = maxpool1d(conv2, 306 - 7 + 1, 256, 3)
    #print(conv2)

    conv3 = conv1d(conv2, weights['wc3'], biases['bc3'], strides=1)
    #print(conv3)

    conv4 = conv1d(conv3, weights['wc4'], biases['bc4'], strides=1)
    #print(conv4)
    conv5 = conv1d(conv4, weights['wc5'], biases['bc5'], strides=1)
    #print(conv5)
    conv6 = conv1d(conv5, weights['wc6'], biases['bc6'], strides=1)
    #print(conv6)
    pool6 = maxpool1d(conv6, 90, 256, 3)
    #print(pool6)
    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    #print(weights['wd1'])
    fc1 = tf.reshape(pool6, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.sigmoid(fc1)
    #print(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)
    #print(fc1)

    fc2 = tf.reshape(fc1, [-1, weights['wd2'].get_shape().as_list()[0]])
    fc2 = tf.add(tf.matmul(fc2, weights['wd2']), biases['bd2'])
    fc2 = tf.nn.sigmoid(fc2)
    #print(fc2)
    fc2 = tf.nn.dropout(fc2, dropout)
    #print(fc2)
    # Output, class prediction
    out = tf.add(tf.matmul(fc2, weights['out']), biases['out'])
    #out = tf.nn.sigmoid(out)
    return out
def multilabel(logits, labels):
    out = tf.nn.softmax(logits)
    out = -tf.reduce_sum(labels * tf.log(out))
    return out
def get_accuracy (logits, labels):
    total_etiquetas = np.sum(labels)
    #print("total etiquetas:", total_etiquetas)
    logits_ = np.copy(logits)
    labels_ = np.copy(labels)
    total = total_etiquetas.astype(int)
    max_x = np.empty(total, dtype=int)
    max_y = np.empty(total, dtype=int)
    #print("labels:", total)
    #print(logits_)
    for i in range(0, total):
        indice = np.argmax(logits_, 1)
        #print(total,i,indice[0])
        max_x[i] = indice[0]
        logits_[0][indice[0]] = 0
        indice = np.argmax(labels_, 1)
        #print(total,i,indice[0])
        max_y[i] = indice[0]
        labels_[0][indice[0]] = 0
    c = np.in1d(max_x,max_y)
    cc = np.where(c == True)[0]
    if len(cc) == 0:
        return 0
    else:
        return 1
# Construct model
pred = conv_net(x, weights, biases, dropout)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

#cost = tf.reduce_mean(multilabel(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
#accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
#accuracy = get_accuracy(logits=pred, labels=y)


init = tf.global_variables_initializer()

In [37]:
batch_x = data[2]
batch_y = data_labels[2]
# Run optimization op (backprop)
batch_x = np.asarray(batch_x).reshape(1, -1).astype(np.float32)
batch_y = np.asarray(batch_y).reshape(1, -1)
ou = conv_net(batch_x, weights, biases, dropout)
prueba = multilabel(logits=ou, labels=batch_y)
print(prueba)
prueba2 = tf.nn.softmax_cross_entropy_with_logits(logits=ou, labels=batch_y)
print(prueba2)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(prueba))
    print(sess.run(prueba2[0]))
    #print(batch_y)
    print(get_accuracy(ou, batch_y))
    print(sess.run(tf.argmax(batch_y, 1)))
    print(sess.run(tf.argmax(ou, 1)))
    print(sess.run(tf.equal(tf.argmax(ou, 1), tf.argmax(batch_y, 1))))
    #print(sess.run(tf.reduce_mean(tf.cast(tf.equal(tf.argmax(ou, 1), tf.argmax(batch_y, 1)), tf.float32))))

Tensor("Neg_17:0", shape=(), dtype=float32)
Tensor("Reshape_415:0", shape=(1,), dtype=float32)
nan
52.9522
('total etiquetas:', 1.0)
('labels:', 1)
(1, 0, 499)
(1, 0, 296)
0
[296]
[654]
[False]


In [7]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    #print(sess.run(weights['wc1']))
    print("init")
    step = 0
    # Keep training until reach max iterations
    epoch = 0
    while epoch < 20:
        print "init epoch" + str (epoch)
        train_data = []
        train_data_labels = []
        test_data = []
        test_data_labels = []
        for i in range(0, 22):
            load_text(i)
            total_train = len(train_data)
            print total_train
            step = 0
            while step < total_train:
                batch_x = train_data[step]
                batch_y = train_data_labels[step]
                # Run optimization op (backprop)
                batch_x = np.asarray(batch_x).reshape(1, -1)
                batch_y = np.asarray(batch_y).reshape(1, -1)
                #print(len(batch_x), len(batch_x[0]))
                sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
                #if epoch % 100 == 0 and epoch >= 100:
                if step % 100 == 0:
                    # Calculate batch loss and accuracy
                    #loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                    #                                                  y: batch_y,
                    #                                                  keep_prob: 1.})
                    loss = sess.run([cost], feed_dict={x: batch_x,
                                                                      y: batch_y,
                                                                      keep_prob: 1.})
                    #print(loss[0])
                    ou = sess.run(pred, feed_dict={x: batch_x, y: batch_y, keep_prob: 1})
                    #print(ou)
                    acc = get_accuracy(ou, batch_y)
                    #print(acc)
                    print "Iter " + str(step) + ", Minibatch Loss= " + \
                          str(loss[0]) + ", Training Accuracy= " + \
                          str(acc)
                step += 1
            train_data = []
            train_data_labels = []
        step = 0
        presition = 0
        total_test = len(test_data)
        print total_test
        while step < total_test:
            batch_x = test_data[step]
            batch_y = test_data_labels[step]
            # Run optimization op (backprop)
            batch_x = np.asarray(batch_x).reshape(1, -1)
            batch_y = np.asarray(batch_y).reshape(1, -1)
            ou = sess.run(pred, feed_dict={x: batch_x, y: batch_y, keep_prob: 1})
            #print(ou)
            acc = get_accuracy(ou, batch_y)
            if acc == 1:
                presition += 1
            step += 1
        print "Presition: " +  str(presition) + "/" + str(total_test) + " " + str(100 * presition / total_test) + " %"
        epoch += 1
    print "Optimization Finished!"

    # Calculate accuracy for 256 mnist test images
    #print "Testing Accuracy:", \
    #    sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
    #                                  y: mnist.test.labels[:256],
    #                                  keep_prob: 1.})

init
init epoch0
Extrayendo textos archivo 0
827
Iter 0, Minibatch Loss= 341.861, Training Accuracy= 0
Iter 100, Minibatch Loss= 72.2341, Training Accuracy= 0
Iter 200, Minibatch Loss= 71.8682, Training Accuracy= 0
Iter 300, Minibatch Loss= 223.518, Training Accuracy= 0
Iter 400, Minibatch Loss= 188.204, Training Accuracy= 1
Iter 500, Minibatch Loss= 54.0541, Training Accuracy= 1
Iter 600, Minibatch Loss= 364.906, Training Accuracy= 0
Iter 700, Minibatch Loss= 21.4814, Training Accuracy= 1
Iter 800, Minibatch Loss= 56.9001, Training Accuracy= 0
Extrayendo textos archivo 1
928
Iter 0, Minibatch Loss= 75.0762, Training Accuracy= 1
Iter 100, Minibatch Loss= 48.0821, Training Accuracy= 0
Iter 200, Minibatch Loss= 1.45434e-05, Training Accuracy= 1
Iter 300, Minibatch Loss= 0.0, Training Accuracy= 1
Iter 400, Minibatch Loss= 175.56, Training Accuracy= 1
Iter 500, Minibatch Loss= 81.9425, Training Accuracy= 1
Iter 600, Minibatch Loss= 26.0492, Training Accuracy= 1
Iter 700, Minibatch Loss= 0.

KeyboardInterrupt: 

In [1]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
get_available_gpus()

[]